# Classification With the Titanic Dataset

In this notebook you will practice a supervised learning problem with the titanic dataset. You will try to predict whether or not a particular passenger lived or died based on other data about that passenger such as age, sex, fare, etc. If you want to continue to explore this dataset, see the Titanic competition on Kaggle:

https://www.kaggle.com/c/titanic

## Imports

In [1]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

## 1. Load and prepare the data

In [2]:
raw_data = sns.load_dataset('titanic')

In [3]:
raw_data.head()

survived  pclass     sex   age  sibsp  parch     fare embarked  class  \
0         0       3    male  22.0      1      0   7.2500        S  Third   
1         1       1  female  38.0      1      0  71.2833        C  First   
2         1       3  female  26.0      0      0   7.9250        S  Third   
3         1       1  female  35.0      1      0  53.1000        S  First   
4         0       3    male  35.0      0      0   8.0500        S  Third   

     who adult_male deck  embark_town alive  alone  
0    man       True  NaN  Southampton    no  False  
1  woman      False    C    Cherbourg   yes  False  
2  woman      False  NaN  Southampton   yes   True  
3  woman      False    C  Southampton   yes  False  
4    man       True  NaN  Southampton    no   True

In [4]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
survived       891 non-null int64
pclass         891 non-null int64
sex            891 non-null object
age            714 non-null float64
sibsp          891 non-null int64
parch          891 non-null int64
fare           891 non-null float64
embarked       889 non-null object
class          891 non-null category
who            891 non-null object
adult_male     891 non-null bool
deck           203 non-null category
embark_town    889 non-null object
alive          891 non-null object
alone          891 non-null bool
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.2+ KB


## 2. Cleaning data

Clean the raw data inplace by doing the following:

* Fill missing values in the age column with its mean.
* Convert bool columns to ints.
* Create a new int valued `child` column that is `1` when the `who` column is `child` and `0` otherwise.
* Drop the `alive`, `deck`, `embarked` and `adult_male` columns.

In [5]:
towns = raw_data.embark_town.value_counts()

In [6]:
# YOUR CODE HERE
raw_data['age'] = raw_data['age'].fillna(value= raw_data['age'].mean())
raw_data[['adult_male','alone']] = raw_data[['adult_male','alone']].astype(int)
raw_data['child'] = raw_data['who'] == 'child'
raw_data['child'] = raw_data['child'].astype(int)
raw_data['embark_town'] = raw_data['embark_town'].fillna(value ='Unknown')
raw_data = raw_data.drop(['embarked', 'adult_male', 'deck', 'alive'], axis = 1)
#raw_data['embark_town']
#raw_data.columns


Get the following tests to pass:

In [7]:
assert raw_data.pclass.isnull().value_counts()[False]==891
assert list(raw_data.sex.unique())==['male', 'female']
assert raw_data.age.isnull().value_counts()[False]==891
assert raw_data.sibsp.isnull().value_counts()[False]==891
assert raw_data.parch.isnull().value_counts()[False]==891
assert raw_data.fare.isnull().value_counts()[False]==891
assert list(raw_data['class'].unique())==['Third', 'First', 'Second']
towns = raw_data.embark_town.value_counts()
assert towns['Southampton']==644
assert towns['Cherbourg']==168
assert towns['Queenstown']==77
assert towns['Unknown']==2
a = raw_data.alone.value_counts()
assert a[0]==354
assert a[1]==537
cc = raw_data.child.value_counts()
assert cc[0]==808
assert cc[1]==83

assert list(raw_data.columns)==['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'class',
       'who', 'embark_town', 'alone', 'child']

Here is a summary of the cleaned data:

In [8]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
survived       891 non-null int64
pclass         891 non-null int64
sex            891 non-null object
age            891 non-null float64
sibsp          891 non-null int64
parch          891 non-null int64
fare           891 non-null float64
class          891 non-null category
who            891 non-null object
embark_town    891 non-null object
alone          891 non-null int64
child          891 non-null int64
dtypes: category(1), float64(2), int64(6), object(3)
memory usage: 77.5+ KB


## 3. Features

Create a feature `DataFrame` named `X` with the numerical columns from the raw dataset:

In [9]:
# YOUR CODE HERE
X = pd.DataFrame(raw_data[['pclass', 'age', 'sibsp', 'parch', 
                           'fare', 'alone', 'child']])

In [10]:
assert list(X.columns)==['pclass', 'age', 'sibsp', 'parch', 'fare', 'alone', 'child']

Use `pandas.get_dummies` to one-hot-encode the `sex`, `class` and `embark_town` columns. Use the `drop_first` argument to drop one of the one-hot encoded columns for each of them. Ese `pandas.concat` to concat the new columns to `X`:

In [11]:
# YOUR CODE HERE
one = pd.get_dummies(raw_data['class'], drop_first=True)
two = pd.get_dummies(raw_data['sex'], drop_first=True)
three = pd.get_dummies(raw_data['embark_town'], drop_first=True)
X = pd.concat([X, two, one, three], axis=1, join_axes=[X.index])

In [12]:
assert list(X.columns)==['pclass', 'age', 'sibsp', 'parch', 'fare', 'alone', 'child', 'male',
       'Second', 'Third', 'Queenstown', 'Southampton', 'Unknown']

Here is the final features we will use:

In [13]:
X.head()

pclass   age  sibsp  parch     fare  alone  child  male  Second  Third  \
0       3  22.0      1      0   7.2500      0      0     1       0      1   
1       1  38.0      1      0  71.2833      0      0     0       0      0   
2       3  26.0      0      0   7.9250      1      0     0       0      1   
3       1  35.0      1      0  53.1000      0      0     0       0      0   
4       3  35.0      0      0   8.0500      1      0     1       0      1   

   Queenstown  Southampton  Unknown  
0           0            1        0  
1           0            0        0  
2           0            1        0  
3           0            1        0  
4           0            1        0

## 4. Target

Create the target vector, `y`, from the `survived` column:

In [14]:
# YOUR CODE HERE
y = raw_data['survived']

In [15]:
yc = y.value_counts()
assert yc[0]==549
assert yc[1]==342

## Train/test split

Use `sklearn.cross_validation.train_test_split` to split your data into a training and test set. Save the resulting data in the variables:

* `Xtrain`
* `Xtest`
* `ytrain`
* `ytest`

Do a train/test split, with 70% of the data used for training and a `random_state=0`:

In [16]:
from sklearn.model_selection import train_test_split

In [17]:
# YOUR CODE HERE
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, train_size = .7, random_state=0)

In [18]:
assert Xtrain.shape==(623,13)
assert ytrain.shape==(623,)
assert Xtest.shape==(268,13)
assert ytest.shape==(268,)

## Gaussian Naive-Bayes classifier

Perform the following steps with the `sklearn.naive_bayes.GaussianNB` classifier:

1. Instantiate the model class
2. Fit the model with the training data
3. Use the model to make predictions about the test data

In [21]:
# YOUR CODE HERE
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(Xtrain, ytrain)
y_model = model.predict(Xtest)

Compute the accuracy of the model:

In [22]:
from sklearn.metrics import accuracy_score
accuracy_score(ytest, y_model)

0.7425373134328358

## Random forest classifier

Perform the following steps with the `sklearn.ensemble.RandomForestClassifier` classifier:

1. Instantiate the model class
2. Fit the model with the training data
3. Use the model to make predictions about the test data
4. Set `random_state=0`

In [26]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import cross_val_score

/opt/conda/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [28]:
# YOUR CODE HERE
model = RandomForestClassifier(random_state=0)
model.fit(Xtrain, ytrain)
ypred = model.predict(Xtest)

Compute the accuracy of the model:

In [29]:
# YOUR CODE HERE
accuracy_score(ytest, ypred)

0.80223880597014929

Use `cross_val_score` to perform k-fold cross validation (`k=10`) with this model:

In [32]:
# YOUR CODE HERE
cross_values = cross_val_score(model, X, y, cv=10)

Compute the average accuracy and its standard deviation:

In [34]:
# YOUR CODE HERE
np.average(cross_values), np.std(cross_values)

(0.80153189195324026, 0.05633558440865833)

Use `sklearn.metrics.confusion_matrix` and Seaborn's `heatmap` to display the confusion matrix for this model:

In [39]:
# YOUR CODE HERE
from sklearn.metrics import confusion_matrix
mat = confusion_matrix(ytest, ypred)
sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False)
plt.xlabel('true label')
plt.ylabel('predicted label');

/opt/conda/lib/python3.5/site-packages/matplotlib/font_manager.py:1297: UserWarning: findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans
  (prop.get_family(), self.defaultFamily[fontext]))


Use the `feature_importances_` attribute of the model to create a `DataFrame` that has two columns:

1. `feature`: the names of the features
2. `importance`: the importances of that feature

Sort by the feature importances.

In [52]:
# YOUR CODE HERE
importances = model.feature_importances_

In [55]:
importances.get_feature_names()

AttributeError: 'numpy.ndarray' object has no attribute 'get_feature_names'

In [47]:
classes = model.n_classes_

In [48]:
classes

2